In [6]:
import pandas as pd
import tensorflow as tf
# import numpy as np

# tf.set_random_seed(1)
# np.random.seed(1)

# 参数，超参数
batch_size = 50
lr = 0.5

def to_onehot(y):
    for i in range(len(y)):
        if y.iloc[i,0]==0:
            y.iloc[i,0] = 'y'
        else:
            y.iloc[i,0] = 'f'
    return pd.get_dummies(y,prefix=y.columns)

# 准备数据
train_data_set = pd.read_csv('train.csv')                   # shape:(4000,1602)
train_data = train_data_set.iloc[0:4000,1:1601]       # shape:(4000,1600)
train_labels = to_onehot(train_data_set.iloc[0:4000,[1601]])           # shape:(4000,1)
print('----')

----


In [7]:
# 先取一小批测试数据看看结果
test_data_set = pd.read_csv('test.csv')                 # shape:(3550,1601) 没有包含正确结果，和训练集不一样
test_data = test_data_set.iloc[0:1000,1:1601]           # shape:(1000,1600)
test_labels = to_onehot(
    pd.read_csv('sample_submit.csv').iloc[0:1000,[1]])          # shape:(1000,1) 一一对应
print('----')

----


In [14]:
# 创建模型
tf_x = tf.placeholder(tf.float32,[None,40*40])          # 每批50个数据，此处暂时不管，为None
x_img = tf.reshape(tf_x,[-1,40,40,1])                   # (batch, height, width, channel)
tf_y = tf.placeholder(tf.float32,[None,2])

## CNN
conv1 = tf.layers.conv2d(           # shape:(40,40,8)
        inputs = x_img,
        filters = 10,               # 图像卷积后的深度
        kernel_size = 5,            # 扫描核5*5大小
        strides = 1,
        padding = 'same',
        activity_regularizer = tf.nn.relu
)                                   # shape:(40,40,10)
pool1 = tf.layers.max_pooling2d(
        inputs = conv1,
        pool_size = 2,
        strides = 2
)                                   # shape:(20,20,10)
conv2 = tf.layers.conv2d(
        inputs = pool1,
        filters = 20,
        kernel_size = 5,
        strides = 1,
        padding = 'same',
        activity_regularizer = tf.nn.relu
)                                   # shape:(20,20,20)
pool2 = tf.layers.max_pooling2d(
        inputs = conv2,
        pool_size = 2,
        strides = 2
)                                   # shape:(10,10,20)
flat_data = tf.reshape(pool2,[-1,10*10*20]) #(10*10*20, )
output = tf.layers.dense(flat_data,2) #用于全连接层，一共输出两种分类个数

loss = tf.losses.softmax_cross_entropy(onehot_labels=tf_y,logits = output)
# loss = tf.reduce_mean(-tf.reduce_sum(tf_y * tf.log(output),reduction_indices=[1]))
train_op = tf.train.GradientDescentOptimizer(lr).minimize(loss)
print('----')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

----


In [15]:
# 计算精度
# 计算精度
accuracy = tf.metrics.accuracy(
        labels = tf.argmax(tf_y,axis = 1),
        predictions = tf.argmax(output,axis = 1)
)[1]

# 重要步骤！！初始化
sess = tf.Session()
#初始化全局和本地变量
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)
print('----')

----


In [16]:
for step in range(1001):
    b_x = train_data.iloc[step*batch_size:(step+1)*batch_size]
    b_y = train_labels.iloc[step*batch_size:(step+1)*batch_size]
    # _,loss_ = sess.run([train_op,loss],feed_dict = {tf_x:b_x,tf_y:b_y})
    sess.run(train_op,feed_dict = {tf_x:b_x,tf_y:b_y})
    if step % 200 ==0:
        accuracy_= sess.run(accuracy,feed_dict = {tf_x:test_data,tf_y:test_labels})
        print('Step:', step,'| test accuracy: %.8f' % accuracy_)
    if step == 1000:
        predictions_2 = tf.argmax(output,axis = 1)
        test_data_ = test_data.iloc[23:167]
        predictions_2_ ,output_= sess.run([predictions_2,output], feed_dict = {tf_x: test_data_})
        print('predictions_2:', predictions_2_,'output:',output_)
        
#当第一次运行完了后，就可以注释掉保存的网络了。
# 改参数可以在这里改，然后保存的模型位置又需要换一个，以免覆盖以前的。
#   保存网络
# saver = tf.train.Saver()
# save_path = saver.save(sess,"my_net/cnn_net.ckpt",write_meta_graph = False)
# print("save_path:",save_path) #打印出来看看

Step: 0 | test accuracy: 0.50000000
Step: 200 | test accuracy: 0.50000000
Step: 400 | test accuracy: 0.50000000
Step: 600 | test accuracy: 0.50000000
Step: 800 | test accuracy: 0.50000000
Step: 1000 | test accuracy: 0.50000000
predictions_2: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] output: [[ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [

In [69]:
b_y = test_labels.iloc[0:batch_size]
print(b_y)
# print(b_y.columns)
b_y.to_csv("b_y.csv",
          index=False,header=False
          )
# b_y_onehot = pd.get_dummies(b_y,prefix=b_y.columns)
# print(b_y_onehot)
# print('b_y[0,0]',train_labels.iloc[0,0])

    y
0   1
1   0
2   1
3   0
4   1
5   0
6   1
7   0
8   1
9   0
10  1
11  0
12  1
13  0
14  1
15  0
16  1
17  0
18  1
19  0
20  1
21  0
22  1
23  0
24  1
25  0
26  1
27  0
28  1
29  0
30  1
31  0
32  1
33  0
34  1
35  0
36  1
37  0
38  1
39  0
40  1
41  0
42  1
43  0
44  1
45  0
46  1
47  0
48  1
49  0


In [46]:
by = pd.read_csv('b_y.csv',names=["y_label"])
print(by)
print(by.columns)

    y_label
0         1
1         0
2         1
3         0
4         1
5         0
6         1
7         0
8         1
9         0
10        1
11        0
12        1
13        0
14        1
15        0
16        1
17        0
18        1
19        0
20        1
21        0
22        1
23        0
24        1
25        0
26        1
27        0
28        1
29        0
30        1
31        0
32        1
33        0
34        1
35        0
36        1
37        0
38        1
39        0
40        1
41        0
42        1
43        0
44        1
45        0
46        1
47        0
48        1
49        0
Index(['y_label'], dtype='object')


In [51]:
print(by.keys())
for name in by.keys():
    print(name,pd.unique(by[name]))

Index(['y_label'], dtype='object')
y_label [1 0]


In [53]:
by_one_hot = pd.get_dummies(by)
print(by_one_hot)

    y_label
0         1
1         0
2         1
3         0
4         1
5         0
6         1
7         0
8         1
9         0
10        1
11        0
12        1
13        0
14        1
15        0
16        1
17        0
18        1
19        0
20        1
21        0
22        1
23        0
24        1
25        0
26        1
27        0
28        1
29        0
30        1
31        0
32        1
33        0
34        1
35        0
36        1
37        0
38        1
39        0
40        1
41        0
42        1
43        0
44        1
45        0
46        1
47        0
48        1
49        0


In [59]:
df = pd.DataFrame({'A': ['a', 'b', 'a'], 'B': ['b', 'a', 'c'], 'C': [1, 2, 3]})
print(df)
pd.get_dummies(df, prefix=['col1', 'col2'])

   A  B  C
0  a  b  1
1  b  a  2
2  a  c  3


,C,col1_a,col1_b,col2_a,col2_b,col2_c
0,1,1,0,0,1,0
1,2,0,1,1,0,0
2,3,1,0,0,0,1


In [7]:
b_x = train_data.iloc[0:batch_size]
print(b_x)  #老是有标签id,p_i_j等，不知输出的时候会不会带上而导致的出错。

    p_0_0  p_0_1  p_0_2  p_0_3  p_0_4  p_0_5  p_0_6  p_0_7  p_0_8  p_0_9  \
0     176    224    164     66    255     79    255    156    118    196   
1     201    113    150    201    132     22    215     99     10    231   
2     255    127    186    255    125    107     36    227     54    125   
3     255    214    179     53    231    242    248     11    146     87   
4     158    134    249    174    140     23     98    194    255    137   
5     255    133    210    220    138      1    172    246     90     43   
6     210     43    146     12    138     26    141    195      7    133   
7      99     67    229     79    255    133    167    255    222    248   
8      44    250    148     10     15    163    206    150    199     10   
9     135    255    224     27    202    129    255    165    171    252   
10     75    177    173    203    187     97    228    113    150    123   
11     86    202    138    194    174    208    255     43     29    155   
12     90   

In [12]:
tf_x_ = tf.placeholder(tf.int32,[None,40*40]) 
x_img_ = tf.reshape(tf_x_,[-1,40,40,1])
print(sess.run(x_img_,feed_dict={tf_x_:b_x}))

[[[[176]
   [224]
   [164]
   ..., 
   [228]
   [225]
   [222]]

  [[ 27]
   [241]
   [ 19]
   ..., 
   [ 43]
   [255]
   [145]]

  [[193]
   [142]
   [ 74]
   ..., 
   [182]
   [213]
   [ 26]]

  ..., 
  [[218]
   [255]
   [140]
   ..., 
   [166]
   [242]
   [178]]

  [[101]
   [  4]
   [211]
   ..., 
   [234]
   [ 49]
   [ 65]]

  [[186]
   [ 25]
   [107]
   ..., 
   [247]
   [240]
   [237]]]


 [[[201]
   [113]
   [150]
   ..., 
   [ 69]
   [117]
   [247]]

  [[187]
   [255]
   [232]
   ..., 
   [ 47]
   [205]
   [ 76]]

  [[248]
   [  7]
   [ 95]
   ..., 
   [228]
   [192]
   [ 29]]

  ..., 
  [[ 93]
   [127]
   [ 33]
   ..., 
   [255]
   [ 40]
   [230]]

  [[255]
   [ 63]
   [ 35]
   ..., 
   [ 10]
   [137]
   [  0]]

  [[118]
   [207]
   [170]
   ..., 
   [131]
   [199]
   [161]]]


 [[[255]
   [127]
   [186]
   ..., 
   [ 48]
   [ 22]
   [139]]

  [[255]
   [133]
   [ 16]
   ..., 
   [ 30]
   [ 20]
   [ 69]]

  [[ 42]
   [ 96]
   [ 46]
   ..., 
   [255]
   [ 64]
   [251]]

  ...

In [6]:
tf_y = tf.placeholder(tf.float32,[None,2])
y = tf.argmax(tf_y,axis = 1)
with tf.Session() as sess1:
    sess1.run(tf.global_variables_initializer())
    print(sess1.run(tf_y,feed_dict={tf_y:train_labels.iloc[12:40]}))
    print(sess1.run(tf.argmax(tf_y,axis = 1),feed_dict={tf_y:train_labels.iloc[12:40]}))
    


[[ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]]
[0 1 0 0 1 1 0 0 0 1 1 0 0 1 1 0 1 0 0 1 1 1 0 1 1 0 1 0]
